<a href="https://colab.research.google.com/github/jdblackstar/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/121A_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [0]:
### YOUR CODE STARTS HERE

import numpy as np
import pandas as pd

column_headers = ['party', 'h-infants', 'water-proj', 'budget', 'phys-freeze', 'salvador-aid',
                  'rel-in-schools', 'anti-sat-test', 'nic-contras', 'mx-missile', 'immigration', 'synfuels-cut',
                  'edu-spend', 'sue-perfund', 'crime', 'd-free', 'export-sa']

df = pd.read_csv('house-votes-84.data')
print(df.shape)
df.head()

(434, 17)


,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
df = pd.read_csv('house-votes-84.data', names = column_headers)

print(df.shape)
df.head()

(435, 17)


,party,h-infants,water-proj,budget,phys-freeze,salvador-aid,rel-in-schools,anti-sat-test,nic-contras,mx-missile,immigration,synfuels-cut,edu-spend,sue-perfund,crime,d-free,export-sa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
df['party'].replace('?', np.nan, inplace= True)
df['h-infants'].replace('?', np.nan, inplace= True)
df['water-proj'].replace('?', np.nan, inplace= True)
df['budget'].replace('?', np.nan, inplace= True)
df['phys-freeze'].replace('?', np.nan, inplace= True)
df['salvador-aid'].replace('?', np.nan, inplace= True)
df['rel-in-schools'].replace('?', np.nan, inplace= True)
df['anti-sat-test'].replace('?', np.nan, inplace= True)
df['nic-contras'].replace('?', np.nan, inplace= True)
df['mx-missile'].replace('?', np.nan, inplace= True)
df['immigration'].replace('?', np.nan, inplace= True)
df['synfuels-cut'].replace('?', np.nan, inplace= True)
df['edu-spend'].replace('?', np.nan, inplace= True)
df['sue-perfund'].replace('?', np.nan, inplace= True)
df['crime'].replace('?', np.nan, inplace= True)
df['d-free'].replace('?', np.nan, inplace= True)
df['export-sa'].replace('?', np.nan, inplace= True)

df.head()

,party,h-infants,water-proj,budget,phys-freeze,salvador-aid,rel-in-schools,anti-sat-test,nic-contras,mx-missile,immigration,synfuels-cut,edu-spend,sue-perfund,crime,d-free,export-sa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [0]:
from scipy import stats

df.sort_values(by=['party'])

df.head(435)

,party,h-infants,water-proj,budget,phys-freeze,salvador-aid,rel-in-schools,anti-sat-test,nic-contras,mx-missile,immigration,synfuels-cut,edu-spend,sue-perfund,crime,d-free,export-sa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,NaN,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,NaN,NaN,NaN,NaN,n,y,y,y,n,y


## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!